In [ ]:
import csv
import numpy as np
import math

In [ ]:
with open('../photo_to_levels.csv') as f:
    food_to_label = []
    for row in csv.DictReader(f, skipinitialspace=True):
        element = {}
        for k, v in row.items():
            if k == "id":
                element['id'] = str(v) + ".jpg"
            elif k == "labels":
                labels_raw = np.array(str(v).split(' '))
                labels = [0] * 9
                labels_int = []
                try:
                    for lb in labels_raw:
                        labels[int(str(lb))] = 1
                        labels_int.append(int(lb))
                except ValueError:
                    print "Failure with value", lb, "labels lenght", len(labels_raw), "content:", v
                element['labels'] = labels
                element['labels_raw'] = labels_int
            else :
                print "No idea what you just passed!"
        
        if len(element['labels_raw']) is not 0:
            food_to_label.append(element)
        else:
            print "Picture", element['id'], "has no labels and is being ignored!"

if len(set([element['id'] for element in food_to_label])) != len(food_to_label):
    print('something\'s wrong!')

In [ ]:
predictions = []

In [ ]:
def combine(n = 8):
    if n < 0:
        return []
    else:
        result = [[n]]
        for e in combine(n-1):
            result.append(e)
            a = []
            a.extend(e)
            a.append(n)
            result.append(a)
        return result

In [ ]:
combinations = [set(e) for e in combine()]

In [ ]:
proportions = []
for combination in combinations:
    l = len([element for element in food_to_label if set(element['labels_raw']) == combination])/float(len(food_to_label))
    proportions.append(l)

In [ ]:
i = 300
print combinations[i], proportions[i]

In [ ]:
with open('./all_classes_prediction.csv') as f:
    allClasses = [{k:v for k,v in row.items()} for row in csv.DictReader(f, skipinitialspace=True)]

In [ ]:
with open('./one_vs_all_class_prediction.csv') as f:
    oneVsAll = [{k:v for k,v in row.items()} for row in csv.DictReader(f, skipinitialspace=True)]

In [ ]:
total = sum([math.exp(float(oneVsAll[0][str(k)])) for k in range(9)])
oneVsAllNorm = []
for element in oneVsAll:
    temp = element.copy()
    for e in range(9):
        temp[str(e)] = math.exp(float(temp[str(e)])) / total
    oneVsAllNorm.append(temp)

In [ ]:
def bestScoring(element, n=3):
    if n is 0:
        return []
    else:
        current = max(element)
        return [current] + bestScoring([e for e in element if e != current], n-1)
    
def getIndexes(values, element):
    return[element.index(v) for v in values]

def getCombinations(array):
    if len(array) is 1:
        return [array]
    else:
        result = [[array[0]]]
        for e in getCombinations(array[1:]):
            result.append(e)
            a = []
            a.extend(e)
            a.append(array[0])
            result.append(a)
        return result

In [ ]:
getCombinations([1,2,3])

In [ ]:
for j in range(4):
    print oneVsAll[j]['id']
    oneVsAllBest = bestScoring([oneVsAllNorm[j][str(v)] for v in range(9)], 2)
    oneVsAllBestIndexes = getIndexes(oneVsAllBest, [oneVsAllNorm[j][str(v)] for v in range(9)])
    combIndex = combinations.index(set(oneVsAllBestIndexes))
    print oneVsAllBestIndexes, "happens at", proportions[combIndex]

    
    allBest = bestScoring([float(allClasses[j][str(v)]) for v in range(9)], 2)
    allBestIndexes = getIndexes(allBest, [float(allClasses[j][str(v)]) for v in range(9)])
    allIndex = combinations.index(set(allBestIndexes))
    print allBestIndexes, "happens at", proportions[allIndex]
    
    for i in range(9):
        print "\t", str(i), "{:.2f}".format(float(oneVsAll[j][str(i)])), "{:.2f}".format(oneVsAllNorm[j][str(i)]), "{:.2f}".format(float(allClasses[j][str(i)]))